目标检测之RCNN算法
===

# 1.图像识别概述
我们知道，在图像分类任务中，一般图片仅仅包含一个物体，我们的任务目标仅仅是根据图片得到图片中物体的类别；但是对于图像识别任务来说，图片内部包含多个物体，我们的任务目标不仅需要对每个物品进行分类，而且还要标识出每个物品的位置。下面就来介绍一下图像识别领域的各种经典算法。目前主流的检测算法可以分为两类：one-state和two-state。前者以YOLO和SSD为代表，后者以RCNN系列为代表。

# 2.RCNN(Region-based Convolutional Neural Networks )
RCNN(Regions with CNN features)是将CNN方法应用到目标检测问题上的一个里程碑，由RBG大神提出，借助CNN良好的特征提取和分类性能，通过RegionProposal方法实现目标检测问题的转化。

## 2.1.RCNN的基本过程
![images](Images/03/02/01_01_001.png)
### 2.1.1.候选区域选择
先将region通过ss(Selective Search)算法检测出来(2k+)。Region Proposal是一类传统的区域提取方法，可以看做不同宽高的滑动窗口，通过窗口滑动获得潜在的目标图像。根据Proposal提取的目标图像进行归一化，作为CNN的标准输入

### 2.1.2.CNN特征提取
标准CNN过程，根据输入进行卷积/池化等操作，得到固定维度的输出

### 2.1.3.分类
根据cnn提取特征丢入svm分类器进行分类(compute score)，得到的就是一个region-bbox以及对应的类别

### 2.1.4.边界回归
由于实际目标会产生多个子区域，旨在对完成分类的前景目标进行精确的定位与合并，避免多个检出。再利用(IoU->nms)得到具体的框，目的防止泛滥，为了精确bbox(nms:非极大值抑制算法).

## 2.2.训练过程
### 2.2.1.预训练
首先需要训练一个完整的alexnet网络，用于生成后续网络的预训练模型

### 2.2.2.数据预处理
输入的数据都是带有bounding box(ground true)信息的图片，对于每张图片都需要做如下处理：
- 使用selectivesearch.selective_search生成若干的bounding box
- 通过算法合并和过滤一些bounding box
- 计算每个bounding box和ground true的IOU，如果大于阀值(一般为0.3)，那么这个bounding box对应的label就是这张图片的label(正样本)；否则对应的label就是背景label(负样本)
- 这样一张图片就可以生成若干的记录，每个记录包括bounding box截取的图片信息、对应的label以及bounding box的坐标信息
- 这些bounding box对应的图片信息，需要resize到224*224(alexnet的输入格式)
- 这样可以生成很多很多图片数据。每批数据应该包含128个实例，其中32个正样本，96个负样本

### 2.2.3.构建新网络
构建一个与之前的alexnet一模一样的网络，只不过需要输出的4096维的特征向量，而不是之前的具体分类个数的特征向量，然后加载之前训练好的模型参数。然后将数据预处理中的图片信息输入到这个网络进行预测，可以得到若干的特征向量，每个特征向量是4096维的。将这些特征向量与对应的label输入到SVM.LinearSVC模型进行训练

### 2.2.4.预测
将需要预测的图片通过selectivesearch.selective_search生成若干的bounding box,然后放入2.3中的新网络生成若干的4096维特征向量，然后将这些特征向量放入已经已经训练好的SVM模型中进行预测，如果结果是背景，那么舍弃，否则这个bounding box就需要画出来。

# 4.bounding-box Regression边框回归
![images](Images/03/02/01_01_002.png)

## 4.1.为何需要边框回归
对于上图，绿色的框表示Ground Truth, 红色的框为Selective Search提取的Region Proposal。那么即便红色的框被分类器识别为飞机，但是由于红色的框定位不准(IoU<0.5)， 那么这张图相当于没有正确的检测出飞机。 如果我们能对红色的框进行微调， 使得经过微调后的窗口跟Ground Truth 更接近， 这样岂不是定位会更准确。 确实，Bounding-box regression 就是用来微调这个窗口的。

## 4.2.边框回归是什么
![images](Images/03/02/01_01_003.png)
对于窗口一般使用四维向量$(x,y,w,h)$来表示,分别表示窗口的中心点坐标和宽高。对于上图, 红色的框P代表原始的Proposal, 绿色的框G代表目标的Ground Truth，我们的目标是寻找一种关系使得输入原始的窗口P经过映射得到一个跟真实窗口G更接近的回归窗口$\hat{G}$。

那么边框回归的目的就是，给定$(P_x,P_y,P_w,P_h)$，寻找一种映射$f$，使得
$$f(P_x,P_y,P_w,P_h)=(\hat{G}_x,\hat{G}_y,\hat{G}_w,\hat{G}_h)$$
并且
$$(\hat{G}_x,\hat{G}_y,\hat{G}_w,\hat{G}_h) \approx (G_x,G_y,G_w,G_h)$$

## 4.3.边框回归的做法
我们知道边框回归的目的就是找到一系列操作，使得P变为$\hat{G}$。那么操作其实就是平移+尺度缩放

### 4.3.1.平移($\Delta{x},\Delta{y}$)
$$\begin{split}
\Delta{x}&=P_wd_x(P) \\
\Delta{y}&=P_hd_y(P) \\
\hat{G}_x&=P_wd_x(P)+P_x \\
\hat{G}_y&=P_hd_y(P)+P_y
\end{split}$$

### 4.3.2.尺度缩放$(S_w,S_h)$
$$\begin{split}
S_w&=exp\[d_w(P)] \\
S_h&=exp\[d_h(P)] \\
\hat{G}_w&=P_wexp\[d_w(P)] \\
\hat{G}_h&=P_hexp\[d_h(P)]
\end{split}$$
边框回归学习就是$d_x(P),d_y(P),d_w(P),d_h(P)$这四个变换。我们需要设计算法那得到这四个映射。线性回归就是给定输入的特征向量X, 学习一组参数W, 使得经过线性回归后的值跟真实值Y(Ground Truth)非常接近. 即$Y \approx WX$

## 4.4.输入与输出
### 4.4.1.输入
边框回归中的输入应该就是通过CNN得到的特征向量，以及Ground Truth

###4.4.2.输出
输出就应该是平移量$t_x,t_y$以及尺度缩放量$t_w,t_h$
$$\begin{split}
t_x&=\frac{G_x-P_x}{P_w} \\
t_y&=\frac{G_y-P_y}{P_h} \\
t_w&=log\frac{G_w}{P_w} \\
t_h=log\frac{G_h}{P_h}
\end{split}$$

## 4.5.损失函数
目标函数表示为
$$d_{\*}(P)=w_{\*}^T\Phi_5(P)$$
其中$\Phi_5(P)$是输入Proposal的特征向量,$w_{\*}$是要学习的参数(*表示 x,y,w,h，也就是每一个变换对应一个目标函数),$d_{\*}(P)$是得到的预测值。损失函数为
$$Loss=\sum_i^N\[t_{\*}^i-\hat{w}_{\*}^T\phi_5(P^i)]^2$$
函数优化目标为：
$$W_{\*}=argmin_{w_{\*}}\sum_i^N\[t_{\*}^i-\hat{w}_{\*}^T\phi_5(P^i)]^2+\lambda||\hat{w}_{\*}||^2$$
利用梯度下降法或者最小二乘法就可以得到$w_\*$

# 5.总结
![images](Images/03/02/01_01_004.png)<br/>
![images](Images/03/02/01_01_005.png)

RCNN存在三个明显的问题
1. 多个候选区域对应的图像需要预先提取，占用较大的磁盘空间；
2. 针对传统CNN需要固定尺寸的输入图像，crop/warp(归一化)产生物体截断或拉伸，会导致输入CNN的信息丢失；
3. 每一个ProposalRegion都需要进入CNN网络计算，上千个Region存在大量的范围重叠，重复的特征提取带来巨大的计算浪费